## Important Dependencies with their required versions


In [6]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb


## Imports and Setup

In [38]:
# from langchain_community.llms import HuggingFaceHub
# from langchain_community.chat_models.huggingface import ChatHuggingFace
import os
import sys
from io import StringIO
import json
from Model import InitializeModel,InitializeGptModel
from GenerateUnitTest.GenerateTest import InitializeTestChain,queryGpt
from GenerateUnitTest.Feedback import *
from utils.PreprocessUtils import *
from utils.FeedbackUtils import *
from utils.LLMUtilis import *
from utils.FuncUtils import *
from DB import *
from dotenv import load_dotenv
load_dotenv()
# from huggingface_hub import notebook_login
# notebook_login()
import pandas as pd    

HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
from CustomThread import CustomThread

def runCode(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    oldstderr = sys.stderr
    redirected_output2 = sys.stderr = StringIO()
    result = ""
    thread = CustomThread()
    try:
        thread.start()
        exec(code, globals())
        result = redirected_output.getvalue()
    except Exception as e:
        # print(repr(e))
        result = repr(e)
    except SystemExit as s:
        # print(repr(s))
        result = redirected_output2.getvalue()
    except KeyboardInterrupt as k:
        result = "timed out"
    thread.stop()
    globals().popitem()
    sys.stdout = old_stdout
    sys.stderr = oldstderr
    # print(result)
    return result

## Initialize Agent

In [39]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     GPTNeoForCausalLM,
# )
# from tokenizers.implementations import ByteLevelBPETokenizer
# tokenizer = ByteLevelBPETokenizer(
#     "vocab.json",
#     "merges.txt",
# )
load_dotenv()
model_id="google/gemma-7b-it"
model_id="mistralai/Mixtral-8x7B-Instruct-v0.1"
max_new_tokens= 6000
if 'Mixtral' in model_id:
    max_new_tokens= 20000

llm, chat_model = InitializeModel(os.environ['HF_TOKEN'], model_id,
                                  max_new_tokens= max_new_tokens)
if 'Mixtral' in model_id:
    GenUnitTestChain= InitializeTestChain(llm)
else:
    GenUnitTestChain= InitializeTestChain(chat_model)

if 'Mixtral' in model_id:
    UnitTestFeedbackChain= InitializeFeedbackChain(llm)
else:
    UnitTestFeedbackChain= InitializeFeedbackChain(chat_model)
    
HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
db= connect_db()
# llm2,llm3=InitializeGptModel(os.environ['OPENAI_API_KEY'])


WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


## Generate Unit Tests from Code

In [43]:
total_test_cases = 0
total_success_test_cases = 0
failed_test_cases = 0
num_failed_examples = 0
num_successful_examples = 0
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]
TotalCases=0
TotalFailedCases=0
incompleteResponses=0
ExcelFile="RunningLogs.xlsx"
JSONFile="RunningLogs.json"
# Check if excel file exists
if not os.path.exists(ExcelFile):
  try:
    # Create a new empty DataFrame with no data
    df = pd.DataFrame()
    df.to_excel(ExcelFile, index=False)
    print(f"File {ExcelFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {ExcelFile}: {e}")
    exit()
if not os.path.exists(JSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(JSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {JSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {JSONFile}: {e}")
    exit()
# try:
with open(ExcelFile):
    df = pd.read_excel(ExcelFile,engine='openpyxl')

try:
  with open(JSONFile, 'r') as f:
    data = json.load(f)
except FileNotFoundError:
  print(f"Error: File {JSONFile} not found!")
  exit()
except json.JSONDecodeError:
  print(f"Error: Invalid JSON format in {JSONFile}!")
  exit()

if not isinstance(data, list):
  print(f"Error: Data in {JSONFile} is not a list. Appending rows requires JSON data as a list of dictionaries!")
  exit()
c=open('cases.txt','a') 
for i in range(0,30):
    # if (i == 10): continue
    print("Running Test Case ",i)
    c.write("Running Test Case "+str(i)+"\n=====================================\n")
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    codes_of_few_shot,test_cases_few_shot=get_few_shots(db,code) 
    #take the most similar few shot other than the code itself
    codes_of_few_shot,test_cases_few_shot=codes_of_few_shot[1:2],test_cases_few_shot[1:2]
    fewShotStr = preprocessStringFewShot(codes_of_few_shot, test_cases_few_shot)
    # print("fewShotStr is :",fewShotStr)
    try:
        unittest=GenUnitTestChain.invoke({"description":description,"code":code,"test_cases_of_few_shot":fewShotStr})#,"test_cases_of_few_shot":fewShotStr
    except Exception as e:
        print("ERROR in invoking GenUnitTestChain")
        print(e)
        c.write("Test Case "+str(i)+"Didn't Run Due to Error\n=====================================\n")
        continue
    ############################################

    ### get the response (unit test) code only without natural language
    unittest_code,isIncompleteResponse = get_code_from_response(unittest['text'])
    if(isIncompleteResponse):
        incompleteResponses+=1
    #print("unittest_code is ",unittest_code)
    unittest_code = replaceUnitTestCall(unittest_code)
    # codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    codeTobeRun = get_running_code(code,unittest_code)
    # print("unittest_code is: \n ",unittest_code)
    # print("run_code is \n",codeTobeRun)
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    print("Feedback is ", feedback)
    
    if(feedbackparsed=='' or feedbackparsed is None):
        #in case of OK
        num_successful_examples +=1
        OKCases+=1
        print(f"Test example {i+1} succeeded" )
        num_of_assertions = get_num_assertions(unittest_code)
        print("Number of Ran Tests : " , num_of_assertions) 
        print("Number of Succeeded Test : ", num_of_assertions)
        print("Number of Succeeded Test : ", 0)
        c.write("Test example "+str(i+1)+" succeeded\n")
        c.write("Number of Ran Tests : "+str(num_of_assertions)+'\n')
        c.write("Number of Succeeded Test : "+str(num_of_assertions)+'\n')

        total_test_cases += num_of_assertions

    else:
         num_failed_examples +=1
         total_cases, failed_cases = get_failed_testcases(feedback)
         print(f"Test example {i+1} failed" )
         print("Number of Ran Tests : " , total_cases) 
         print("Number of failed Tests : ", failed_cases)
         print("Number of Succeeded Test : ", total_cases - failed_cases)
         c.write("Test example "+str(i+1)+" failed\n")
         c.write("Number of Ran Tests : "+str(total_cases)+'\n')
         c.write("Number of failed Tests : "+str(failed_cases)+'\n')
         c.write("Number of Succeeded Test : "+str(total_cases - failed_cases)+'\n')
         total_test_cases += total_cases
         failed_test_cases+= failed_cases
        

    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    new_row=pd.DataFrame({'CaseNumber':i,'Description':description,'Code':code,'GeneratedCode':unittest_code,'Feedback':feedbackparsed,"FullFeedback":feedback},index=[0])
    df = df.append(new_row, ignore_index=True)
    # Save the updated DataFrame back to the excel file using 'openpyxl' engine for writing
    with pd.ExcelWriter(ExcelFile, engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:
        # df.to_excel(writer, sheet_name=df.sheet_name.tolist()[0], index=False)  # Assuming only one sheet
        df.to_excel(writer, index=False)  # Assuming only one sheet
    jsondata=df.to_dict(orient='records')
    with open(JSONFile, 'w') as f:
        json.dump(jsondata, f, indent=4)

c.close()
#dump data into xlsx file
# resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
# #resDF.to_excel("GeneratedFeedback.xlsx",index=False)
# resDF.to_json("GeneratedFeedback.json",orient='records')
# print("OKCases are ",OKCases)
print('\n End of Test Cases \n')
print("Total succeeded examples : " ,num_successful_examples)
print("Total failed examples : ", num_failed_examples)
print("Total  testcases : " ,total_test_cases)
print("Total succeeded testcases : ",total_test_cases- failed_test_cases)
print("Total failed testcases : ",failed_test_cases)
with open('Res.txt', 'w') as f:
    f.write('Total succeeded examples : ' + str(num_successful_examples) + '\n')
    f.write('Total failed examples : ' + str(num_failed_examples) + '\n')
    f.write('Total  testcases : ' + str(total_test_cases) + '\n')
    f.write('Total succeeded testcases : ' + str(total_test_cases- failed_test_cases) + '\n')
    f.write('Total failed testcases : ' + str(failed_test_cases) + '\n')

print("Incomplete Responses are ",incompleteResponses)

File RunningLogs.xlsx created successfully!
File RunningLogs.json created successfully!
Running Test Case  1
Feedback is  .F.....
FAIL: test_close_elements_at_end (__main__.TestHasCloseElements.test_close_elements_at_end)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 30, in test_close_elements_at_end
AssertionError: False is not true

----------------------------------------------------------------------
Ran 7 tests in 0.000s

FAILED (failures=1)

Test example 1 failed
Number of Ran Tests :  7
Number of failed Tests :  1
Number of Succeeded Test :  6
Running Test Case  2
Feedback is  ..F.F.F
FAIL: test_mixed_content (__main__.TestSeparateParenGroups.test_mixed_content)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 46, in test_mixed_content
AssertionError: Lists differ: ['()'] != ['(world)']

First differing element 0:
'(

C:\Users\DELL\AppData\Local\Temp\ipykernel_13848\128043486.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_13848\128043486.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Running Test Case  3
Feedback is  FF.FF..
FAIL: test_truncate_negative_float_non_zero_decimal (__main__.TestTruncateNumber.test_truncate_negative_float_non_zero_decimal)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 34, in test_truncate_negative_float_non_zero_decimal
AssertionError: 0.40000000000000036 != 0.4

FAIL: test_truncate_negative_float_zero_decimal (__main__.TestTruncateNumber.test_truncate_negative_float_zero_decimal)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 31, in test_truncate_negative_float_zero_decimal
AssertionError: 0.9900000000000002 != 0.01

FAIL: test_truncate_positive_float_non_zero_decimal (__main__.TestTruncateNumber.test_truncate_positive_float_non_zero_decimal)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 25

C:\Users\DELL\AppData\Local\Temp\ipykernel_13848\128043486.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Running Test Case  4
Feedback is  ..F....
FAIL: test_last_negative_operation (__main__.TestBelowZero.test_last_negative_operation)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 31, in test_last_negative_operation
AssertionError: False is not true

----------------------------------------------------------------------
Ran 7 tests in 0.000s

FAILED (failures=1)

Test example 4 failed
Number of Ran Tests :  7
Number of failed Tests :  1
Number of Succeeded Test :  6


...

C:\Users\DELL\AppData\Local\Temp\ipykernel_13848\128043486.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Feedback is  .......
----------------------------------------------------------------------
Ran 7 tests in 0.000s

OK

Test example 29 succeeded
Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Succeeded Test :  0
Running Test Case  30


C:\Users\DELL\AppData\Local\Temp\ipykernel_13848\128043486.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Feedback is  .....FF
FAIL: test_prefix_at_end (__main__.TestFilterByPrefix.test_prefix_at_end)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 29, in test_prefix_at_end
AssertionError: Lists differ: [] != ['array']

Second list contains 1 additional elements.
First extra element 0:
'array'

- []
+ ['array']

FAIL: test_prefix_case_insensitive (__main__.TestFilterByPrefix.test_prefix_case_insensitive)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<string>", line 35, in test_prefix_case_insensitive
AssertionError: Lists differ: ['array'] != ['AbC', 'array']

First differing element 0:
'array'
'AbC'

Second list contains 1 additional elements.
First extra element 1:
'array'

- ['array']
+ ['AbC', 'array']

----------------------------------------------------------------------
Ran 7 tests in 0.000s

FAILED (failures=2)

Test example 30 failed

C:\Users\DELL\AppData\Local\Temp\ipykernel_13848\128043486.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


## Feedback to LLM for regenerating again

In [35]:
print(Cases.iloc[9]["Feedback"]=="")
print(pd.isna(Cases.iloc[9]["Feedback"]))

True
False


In [37]:
total_test_cases = 0
total_success_test_cases = 0
failed_test_cases = 0
num_failed_examples = 0
num_successful_examples = 0
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]
TotalCases=0
TotalFailedCases=0
incompleteResponses=0
ExcelFile="RunningLogs.xlsx"
JSONFile="RunningLogs.json"

OldJsonFile="Mixtral-1Shot/RunningLogs.json"
Cases=pd.read_json(OldJsonFile)
if not os.path.exists(ExcelFile):
  try:
    # Create a new empty DataFrame with no data
    df = pd.DataFrame()
    df.to_excel(ExcelFile, index=False)
    print(f"File {ExcelFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {ExcelFile}: {e}")
    exit()
if not os.path.exists(JSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(JSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {JSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {JSONFile}: {e}")
    exit()
# try:
with open(ExcelFile):
    df = pd.read_excel(ExcelFile,engine='openpyxl')

try:
  with open(JSONFile, 'r') as f:
    data = json.load(f)
except FileNotFoundError:
  print(f"Error: File {JSONFile} not found!")
  exit()
except json.JSONDecodeError:
  print(f"Error: Invalid JSON format in {JSONFile}!")
  exit()

if not isinstance(data, list):
  print(f"Error: Data in {JSONFile} is not a list. Appending rows requires JSON data as a list of dictionaries!")
  exit()
c=open('cases.txt','a') 
count=0
for i in range(len(Cases)):
    currDescription=Cases.iloc[i]['Description']
    currCode=Cases.iloc[i]['Code']
    currGeneratedCode=Cases.iloc[i]['GeneratedCode']
    currFeedback=Cases.iloc[i]['Feedback']
    # no feedback means testcase passed so don't run it again
    if pd.isna(currFeedback) or currFeedback=="":
       count+=1
       print("case",i," is already passed")
       continue
print("count is ",count)
#     try:
#         GenerationPostFeedback=UnitTestFeedbackChain.invoke({"description":currDescription,"code":currCode,"UnitTests":currGeneratedCode,"Feedback":currFeedback})
#         print("Full Generation is :",GenerationPostFeedback)
#     except Exception as e:
#         print("ERROR in invoking UnitTestFeedbackChain")
#         print(e)
#         continue
#     newUnitTestCode,isIncompleteResponse = get_code_from_response(GenerationPostFeedback['text'])
#     if(isIncompleteResponse):
#         incompleteResponses+=1
#     unittest_code = replaceUnitTestCall(newUnitTestCode)
#     codeTobeRun = get_running_code(currCode,unittest_code)
#     feedback = runCode(code=codeTobeRun)
#     feedbackparsed=get_feedback_from_run(feedback)

#     if(feedbackparsed==''):
#         #in case of OK
#         num_successful_examples +=1
#         OKCases+=1
#         print(f"Test example {i+1} succeeded" )
#         num_of_assertions = get_num_assertions(unittest_code)
#         print("Number of Ran Tests : " , num_of_assertions) 
#         print("Number of Succeeded Test : ", num_of_assertions)
#         print("Number of Succeeded Test : ", 0)
#         c.write("Test example "+str(i+1)+" succeeded\n")
#         c.write("Number of Ran Tests : "+str(num_of_assertions)+'\n')
#         c.write("Number of Succeeded Test : "+str(num_of_assertions)+'\n')

#         total_test_cases += num_of_assertions

#     else:
#          num_failed_examples +=1
#          total_cases, failed_cases = get_failed_testcases(feedback)
#          print(f"Test example {i+1} failed" )
#          print("Number of Ran Tests : " , total_cases) 
#          print("Number of failed Tests : ", failed_cases)
#          print("Number of Succeeded Test : ", total_cases - failed_cases)
#          c.write("Test example "+str(i+1)+" failed\n")
#          c.write("Number of Ran Tests : "+str(total_cases)+'\n')
#          c.write("Number of failed Tests : "+str(failed_cases)+'\n')
#          c.write("Number of Succeeded Test : "+str(total_cases - failed_cases)+'\n')
#          total_test_cases += total_cases
#          failed_test_cases+= failed_cases
    
#     descriptions.append(currDescription)
#     codes.append(currCode)
#     res_codes.append(unittest_code)
#     feedbacks.append(feedbackparsed)  #feedbackparsed
#     new_row=pd.DataFrame({'CaseNumber':i,'Description':currDescription,'Code':currCode,'GeneratedCode':unittest_code,'Feedback':feedbackparsed},index=[0])
#     df = df.append(new_row, ignore_index=True)
#     # Save the updated DataFrame back to the excel file using 'openpyxl' engine for writing
#     with pd.ExcelWriter(ExcelFile, engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:
#         # df.to_excel(writer, sheet_name=df.sheet_name.tolist()[0], index=False)  # Assuming only one sheet
#         df.to_excel(writer, index=False)  # Assuming only one sheet
#     jsondata=df.to_dict(orient='records')
#     with open(JSONFile, 'w') as f:
#         json.dump(jsondata, f, indent=4)


# c.close()
# print('\n End of Test Cases \n')
# print("Total succeeded examples : " ,num_successful_examples)
# print("Total failed examples : ", num_failed_examples)
# print("Total  testcases : " ,total_test_cases)
# print("Total succeeded testcases : ",total_test_cases- failed_test_cases)
# print("Total failed testcases : ",failed_test_cases)
# with open('Res.txt', 'w') as f:
#     f.write('Total succeeded examples : ' + str(num_successful_examples) + '\n')
#     f.write('Total failed examples : ' + str(num_failed_examples) + '\n')
#     f.write('Total  testcases : ' + str(total_test_cases) + '\n')
#     f.write('Total succeeded testcases : ' + str(total_test_cases- failed_test_cases) + '\n')
#     f.write('Total failed testcases : ' + str(failed_test_cases) + '\n')

# print("Incomplete Responses are ",incompleteResponses)


case 1  is already passed
case 2  is already passed
case 5  is already passed
case 9  is already passed
case 17  is already passed
case 18  is already passed
case 22  is already passed
case 23  is already passed
count is  8


New row appended to your_file.json successfully!


C:\Users\DELL\AppData\Local\Temp\ipykernel_19324\1510761095.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


## GPT MODEL

In [ ]:
introCode = """
import sys
sys.setrecursionlimit(100)
"""
import time
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]

for i in range(0,10):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    code_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessStringFewShot(code_of_few_shot, test_cases_few_shot)

    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model
    #try gpt model
    unittest=queryGpt(llm2,description,code,fewshots=False,test_cases_of_few_shot=fewShotStr)['choices'][0]['message']['content']
    print("gpt_response is ",unittest)

    
    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model

    ### get the response (unit test) code only without natural language
    unittest_code = get_code_from_response(unittest,funcDefiniton)
    unittest_code = replaceUnitTestCall(unittest_code)
    ##############run code with the original and unit test concatenated #######################
    codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    print("run_code is")
    print(codeTobeRun)
    # print("------------------------------lolooloolololo ------------------------------------")
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    if(feedbackparsed==''):
        #in case of OK
        OKCases+=1
        # feedbackparsed=""
        # for i,line in enumerate(feedback.split('\n')):
        #     if i>1:
        #         feedbackparsed+=line+'\n'
    #print(feedbackparsed)
    print(feedback)
    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    time.sleep(20)

#dump data into xlsx file
resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
#resDF.to_excel("GeneratedFeedbackGPT.xlsx",index=False)
resDF.to_json("GeneratedFeedbackGPT.json",orient='records')
print("OKCases are ",OKCases)
print("Total Cases are ",len(descriptions))
with open('ResGPT.txt', 'w') as f:
    f.write("OKCases are "+str(OKCases)+"\n")
    f.write("Total Cases are "+str(len(descriptions))+"\n")


## Bug Fixing

In [ ]:
BugRepair_template_str = """Act as a python expert in bug fixing and assume that I need to fix all errors perfectly.
I am going to give you the bugs encountered due to running test cases then after
that I will give you the code under test with bugs so that you fix it, the line numbers of
the error correspond to the line that the error occured in the code provided.
The following is the code under test:
```python
{code}```

The following are the bugs and test cases corresponding to the bugs:
```python
{TestcaseCode}```

The result after running these test cases on an interpreter:
{InterpreterFeedback}

Change either the code or the test cases according to the feedback of the interpreter.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
"""
InterpreterFeedback=feedbackparsed

BugRepair_Template = PromptTemplate(
    template=BugRepair_template_str,
    input_variables=["code","TestcaseCode","InterpreterFeedback"],
    verbose=True
    )

BugFixChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=BugRepair_Template
)

res=BugFixChain.invoke({
                             "code":code,
                             "TestcaseCode":res_code,
                             "InterpreterFeedback":InterpreterFeedback
                             })
print(res['text'])

## input code preprocessor

In [ ]:
#if the input code is without description, we want to generate description
Generate_description_Template = """Act as a python expert, Describe the purpose and functionality of the following Python code.

```python
{code}```
The output should be like that:
The provided Python code aims to [explain purpose].
In summary, [concise summary of functionality ].
Examples: [3 or 4 input output examples]
"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
Generate_description_PromptTemplate = PromptTemplate(
    template=Generate_description_Template,
    input_variables=["code"],
    verbose=True
    )

GenDescriptionChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_description_PromptTemplate
)
res=GenDescriptionChain.invoke({
                             "code":code
                             })
print(res['text'])